In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sn
import folium
import math
import json
from branca.colormap import LinearColormap
from branca.element import MacroElement
from jinja2 import Template


In [2]:
intercountry_df = pd.read_csv('../Data_sinan/intercountry_tone_count_table_with_EVENTCODE',\
                              dtype={'EventRootCode': np.str, 'EventCode': np.str} \
                             )
country_info_df = pd.read_csv('../Data_sinan/countryinfo.csv')
world_borders = json.load(open('../Data/world_borders.topojson.json'))
fips_codes = pd.read_csv('../Data/fipscountries.csv')
fips_codes = fips_codes[['fips','country']]


In [3]:
population = country_info_df[['fips', 'population']]

In [4]:
intercountry_df = intercountry_df[pd.notnull(intercountry_df['LocEvent'])]
intercountry_df = (intercountry_df[(intercountry_df["EventRootCode"]!='--') & (intercountry_df["EventCode"]!='--')])


In [5]:
intercountry_df = fips_codes.merge(intercountry_df, left_on='fips', right_on='LocEvent', how='inner')
intercountry_df.loc[intercountry_df['country']=='United States', 'country'] = 'United States of America'
intercountry_df.loc[intercountry_df['country']=='Serbia', 'country'] = 'Republic of Serbia'


In [6]:
intercountry_nb_mentions_df = intercountry_df[['country', 'EventRootCode', 'EventCode', 'NumberOfInterMentions', 'fips']]
countries_protests = intercountry_nb_mentions_df[intercountry_nb_mentions_df['EventRootCode'] == '14']
countries_protests = countries_protests.groupby(['country', 'fips'], as_index=False).NumberOfInterMentions.sum()


In [7]:
leadership_change_protests_df = intercountry_df[(intercountry_df['EventCode']=='1411') |
                                                (intercountry_df['EventCode']=='1431') |
                                                (intercountry_df['EventCode']=='1441') |
                                                (intercountry_df['EventCode']=='1451') 
                                               ]

leadership_change_protests_df = leadership_change_protests_df[['country', 'fips', 'NumberOfInterMentions']]
leadership_change_protests_df = leadership_change_protests_df.groupby(['country', 'fips'], as_index=False).NumberOfInterMentions.sum()
leadership_change_protests_df.head()


,country,fips,NumberOfInterMentions
0,Afghanistan,AF,324
1,Albania,AL,110
2,Algeria,AG,6
3,Angola,AO,18
4,Antigua and Barbuda,AC,2


In [8]:
relative_leadership_change_protests_df = countries_protests.merge(leadership_change_protests_df, 
                                                                  on='country', 
                                                                  how='outer'
                                                                 ) \
                                                            .drop(columns=['fips_y']) \
                                                            .rename(columns={"fips_x": "fips", \
                                                                             "NumberOfInterMentions_x": "NumberOfProtests", \
                                                                             "NumberOfInterMentions_y": "NumberOfLeadershipChangeProtests"} \
                                                                   ) \
                                                            .merge(population, on='fips', how='inner') 

relative_leadership_change_protests_df.head()

,country,fips,NumberOfProtests,NumberOfLeadershipChangeProtests,population
0,Afghanistan,AF,55915,324.0,29121286
1,Albania,AL,61551,110.0,2986952
2,Algeria,AG,14912,6.0,34586184
3,American Samoa,AQ,92,NaN,57881
4,Andorra,AN,400,NaN,84000


In [9]:
relative_leadership_change_protests_df['Relative number of protest by population'] = relative_leadership_change_protests_df['NumberOfProtests'] \
                                                                            /relative_leadership_change_protests_df['population']

relative_leadership_change_protests_df['Leadership: Relative by total nb of protest'] = relative_leadership_change_protests_df['NumberOfLeadershipChangeProtests'] \
                                                                            /relative_leadership_change_protests_df['NumberOfProtests']

relative_leadership_change_protests_df['Leadership: Relative by population'] = relative_leadership_change_protests_df['NumberOfLeadershipChangeProtests'] \
                                                                  /relative_leadership_change_protests_df['population']

#Population is 0 so the division is inf
relative_leadership_change_protests_df['Relative number of protest by population'].replace(np.inf, 0, inplace=True)

relative_leadership_change_protests_df.head()
                                                

,country,fips,NumberOfProtests,NumberOfLeadershipChangeProtests,population,Relative number of protest by population,Leadership: Relative by total nb of protest,Leadership: Relative by population
0,Afghanistan,AF,55915,324.0,29121286,0.001920,0.005795,1.112588e-05
1,Albania,AL,61551,110.0,2986952,0.020607,0.001787,3.682684e-05
2,Algeria,AG,14912,6.0,34586184,0.000431,0.000402,1.734797e-07
3,American Samoa,AQ,92,NaN,57881,0.001589,NaN,NaN
4,Andorra,AN,400,NaN,84000,0.004762,NaN,NaN


In [10]:
relative_leadership_change_protests_df.loc[relative_leadership_change_protests_df['country'] == 'Germany']

,country,fips,NumberOfProtests,NumberOfLeadershipChangeProtests,population,Relative number of protest by population,Leadership: Relative by total nb of protest,Leadership: Relative by population
76,Germany,GM,205536,276.0,81802257,0.002513,0.001343,0.000003


In [11]:
relative_leadership_change_protests_df.loc[relative_leadership_change_protests_df['country'] == 'France']

,country,fips,NumberOfProtests,NumberOfLeadershipChangeProtests,population,Relative number of protest by population,Leadership: Relative by total nb of protest,Leadership: Relative by population
70,France,FR,284695,396.0,64768389,0.004396,0.001391,0.000006


In [12]:
relative_leadership_change_protests_df.loc[relative_leadership_change_protests_df['country'] == 'Switzerland']

,country,fips,NumberOfProtests,NumberOfLeadershipChangeProtests,population,Relative number of protest by population,Leadership: Relative by total nb of protest,Leadership: Relative by population
203,Switzerland,SZ,35811,100.0,7581000,0.004724,0.002792,0.000013


In [13]:
relative_leadership_change_protests_df.loc[relative_leadership_change_protests_df['country'] == 'United States of America']

,country,fips,NumberOfProtests,NumberOfLeadershipChangeProtests,population,Relative number of protest by population,Leadership: Relative by total nb of protest,Leadership: Relative by population
223,United States of America,US,2585806,4258.0,310232863,0.008335,0.001647,0.000014


In [14]:
relative_leadership_change_protests_df.loc[relative_leadership_change_protests_df['country'] == 'India']

,country,fips,NumberOfProtests,NumberOfLeadershipChangeProtests,population,Relative number of protest by population,Leadership: Relative by total nb of protest,Leadership: Relative by population
93,India,IN,406259,2236.0,1173108018,0.000346,0.005504,0.000002


In [15]:
relative_leadership_change_protests_df.loc[relative_leadership_change_protests_df['country'] == 'China']

,country,fips,NumberOfProtests,NumberOfLeadershipChangeProtests,population,Relative number of protest by population,Leadership: Relative by total nb of protest,Leadership: Relative by population
43,China,CH,168832,281.0,1330044000,0.000127,0.001664,2.112712e-07


In [16]:
def color_function(feature, data, color_fun):
    value = data.get(feature['properties']['name'])
    if value is None:
        return '#8c8c8c' # MISSING -> gray
    if math.isnan(value):
        return '#8c8c8c' # MISSING -> gray
    else:
        return color_fun(value)

In [17]:
class BindColormap(MacroElement):
    """Binds a colormap to a given layer.

    Parameters
    ----------
    colormap : branca.colormap.ColorMap
        The colormap to bind.
    """
    def __init__(self, layer, colormap):
        super(BindColormap, self).__init__()
        self.layer = layer
        self.colormap = colormap
        self._template = Template(u"""
        {% macro script(this, kwargs) %}
            {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
            {{this._parent.get_name()}}.on('overlayadd', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
                }});
            {{this._parent.get_name()}}.on('overlayremove', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'none';
                }});
        {% endmacro %}
        """)  # noqa

In [18]:
world_map = folium.Map(tiles='Mapbox bright', zoom_start=5.5)

# Defining a style function to color the map and the borders
style_func=lambda feature: {'fillColor': '#990033','color' : 'black','weight' : 2 }

# Using a GeoJsonTooltip to display cantons' name when mouse is over them
tool=folium.features.GeoJsonTooltip(fields=['name'], aliases=['Country:'], sticky=True, \
                           style="font-family: Arial; color: black;", \
                           opacity=0.8, \
                           direction='top' \
                          )

# Load border information on the map
folium.TopoJson(world_borders,\
                object_path='objects.countries1',\
                style_function=style_func,\
                tooltip=tool, \
                overlay=False \
               ) \
      .add_to(world_map)

In [26]:
protest_dict = relative_leadership_change_protests_df.set_index('country')['NumberOfProtests'].to_dict()
protest_dict

{'Afghanistan': 55915,
 'Albania': 61551,
 'Algeria': 14912,
 'American Samoa': 92,
 'Andorra': 400,
 'Angola': 10916,
 'Anguilla': 69,
 'Antarctica': 462,
 'Antigua and Barbuda': 209,
 'Argentina': 48420,
 'Armenia': 40698,
 'Aruba': 203,
 'Australia': 127447,
 'Austria': 21988,
 'Azerbaijan': 11272,
 'Bahamas': 2365,
 'Bahrain': 37597,
 'Bangladesh': 54809,
 'Barbados': 780,
 'Belarus': 21691,
 'Belgium': 47701,
 'Belize': 1131,
 'Benin': 3038,
 'Bermuda': 611,
 'Bhutan': 626,
 'Bolivia': 18622,
 'Bosnia and Herzegovina': 3978,
 'Botswana': 1242,
 'Brazil': 124682,
 'British Indian Ocean Territory': 26,
 'British Virgin Islands': 344,
 'Brunei': 2384,
 'Bulgaria': 26910,
 'Burkina Faso': 15521,
 'Burundi': 23690,
 'Cambodia': 12886,
 'Cameroon': 8652,
 'Canada': 99126,
 'Cape Verde': 164,
 'Cayman Islands': 251,
 'Central African Republic': 4884,
 'Chad': 3044,
 'Chile': 30865,
 'China': 168832,
 'Christmas Island': 68,
 'Colombia': 34143,
 'Comoros': 280,
 'Cook Islands': 107,
 'Cos

In [19]:
color_scale_protest = LinearColormap(['yellow','red'], \
                                     vmin = min(protest_dict.values()), \
                                     vmax = max(protest_dict.values()), \
                                     caption='Protests over the world'\
                                    )

In [20]:
map1 = folium.TopoJson(
    world_borders,
    object_path='objects.countries1',
    style_function=lambda feature: {
        'fillColor': color_function(feature, protest_dict, color_scale_protest),
        'color' : 'black',
        'weight' : 1,
        'dashArray' : '5, 5',
        'fillOpacity' : 0.8
        },
    tooltip=folium.features.GeoJsonTooltip(fields=['name'],aliases=['Country: '], sticky=True, style="font-family: Arial; color: black;", \
                           opacity=0.8, direction='top'),
    name = 'Protests over the world',
    )

In [21]:
protest_by_population_dict = relative_leadership_change_protests_df.set_index('country')['Relative number of protest by population'] \
                                                                   .to_dict()
color_scale_protest_by_population = LinearColormap(['yellow','red'], \
                                                   vmin = min(protest_by_population_dict.values()), \
                                                   vmax = max(protest_by_population_dict.values()), \
                                                   caption='Protests normalized by the population'\
                                                  )

map2 = folium.TopoJson(
    world_borders,
    object_path='objects.countries1',
    style_function=lambda feature: {
        'fillColor': color_function(feature, protest_by_population_dict, color_scale_protest_by_population),
        'color' : 'black',
        'weight' : 1,
        'dashArray' : '5, 5',
        'fillOpacity' : 0.8
        },
    tooltip=folium.features.GeoJsonTooltip(fields=['name'], aliases=['Canton: '], sticky=True, style="font-family: Arial; color:black;",
                                  opacity=0.8, direction='top'),
    name = 'Protests normalized by population',
    #show = False
    )


In [22]:
leadership_dict = relative_leadership_change_protests_df.set_index('country')['NumberOfLeadershipChangeProtests'] \
                                                        .to_dict()
color_scale_leadership = LinearColormap(['yellow','red'], \
                                        vmin = min(leadership_dict.values()), \
                                        vmax = max(leadership_dict.values()), \
                                        caption='Leadership change protests over the world'\
                                       )

map3 = folium.TopoJson(
    world_borders,
    object_path='objects.countries1',
    style_function=lambda feature: {
        'fillColor': color_function(feature, leadership_dict, color_scale_leadership),
        'color' : 'black',
        'weight' : 1,
        'dashArray' : '5, 5',
        'fillOpacity' : 0.8
        },
    tooltip=folium.features.GeoJsonTooltip(fields=['name'], aliases=['Canton: '], sticky=True, style="font-family: Arial; color:black;",
                                  opacity=0.8, direction='top'),
    name = 'Protests for leadership change',
    #show = False
    )


In [23]:
leadership_by_protest_dict = relative_leadership_change_protests_df.set_index('country')['Leadership: Relative by total nb of protest'] \
                                                                   .to_dict()
color_scale_leadership_by_protest = LinearColormap(['yellow','red'], \
                                                   vmin = min(leadership_by_protest_dict.values()), \
                                                   vmax = max(leadership_by_protest_dict.values()), \
                                                   caption='Leadership change protests normalized by the number of protests'\
                                                  )

map4 = folium.TopoJson(
    world_borders,
    object_path='objects.countries1',
    style_function=lambda feature: {
        'fillColor': color_function(feature, leadership_by_protest_dict, color_scale_leadership_by_protest),
        'color' : 'black',
        'weight' : 1,
        'dashArray' : '5, 5',
        'fillOpacity' : 0.8
        },
    tooltip=folium.features.GeoJsonTooltip(fields=['name'], aliases=['Canton: '], sticky=True, style="font-family: Arial; color:black;",
                                  opacity=0.8, direction='top'),
    name = 'Protests for leadership change normalized by the number of protest',
    #show = False
    )

In [24]:
leadership_by_population_dict = relative_leadership_change_protests_df.set_index('country')['Leadership: Relative by population'] \
                                                                   .to_dict()

color_scale_leadership_by_population = LinearColormap(['yellow','red'], \
                                                      vmin = min(leadership_by_population_dict.values()), \
                                                      vmax = max(leadership_by_population_dict.values()), \
                                                      caption='Leadership change protests normalized by the population'\
                                                     )


map5 = folium.TopoJson(
    world_borders,
    object_path='objects.countries1',
    style_function=lambda feature: {
        'fillColor': color_function(feature, leadership_by_population_dict, color_scale_leadership_by_population),
        'color' : 'black',
        'weight' : 1,
        'dashArray' : '5, 5',
        'fillOpacity' : 0.8
        },
    tooltip=folium.features.GeoJsonTooltip(fields=['name'], aliases=['Canton: '], sticky=True, style="font-family: Arial; color:black;",
                                  opacity=0.8, direction='top'),
    name = 'Protests for leadership change normalized by the population',
    #show = False
    )

In [25]:
world_map.add_child(map1).add_child(map2).add_child(map3).add_child(map4).add_child(map5)

world_map.add_child(folium.map.LayerControl())

world_map.add_child(color_scale_protest)\
         .add_child(color_scale_protest_by_population)\
         .add_child(color_scale_leadership)\
         .add_child(color_scale_leadership_by_protest)\
         .add_child(color_scale_leadership_by_population)\

world_map.add_child(BindColormap(map1, color_scale_protest)) \
         .add_child(BindColormap(map2, color_scale_protest_by_population))\
         .add_child(BindColormap(map3, color_scale_leadership))\
         .add_child(BindColormap(map4, color_scale_leadership_by_protest))\
         .add_child(BindColormap(map5, color_scale_leadership_by_population))

world_map

In [ ]:
happiness_df = pd.read_excel('../Data_sinan/WHR2018Chapter2OnlineData.xls')

In [ ]:
happiness_df = happiness_df[happiness_df['year']==2016][['country','Confidence in national government', 'Democratic Quality']]

In [ ]:
#min(happiness_df['Confidence in national government'])
happiness_df['difference'] = happiness_df['Democratic Quality'] - happiness_df['Confidence in national government'] 

In [ ]:
happiness_df.loc[happiness_df['country']=='United States']

In [ ]:
happiness_df.loc[happiness_df['country']=='India']

In [ ]:
happiness_df.loc[happiness_df['country']=='Iran']

In [ ]:
happiness_df.loc[happiness_df['country']=='Russia']

In [ ]:
happiness_df.loc[happiness_df['country']=='Kazakhstan']

In [ ]:
happiness_df.loc[happiness_df['country']=='Mali']

In [28]:
happiness_df.loc[happiness_df['country']=='North Korea']

,country,Confidence in national government,Democratic Quality,difference
